In [1]:
import os
str1 = ''
file_list = []
os.chdir(r"C:\Users\ThinkPad\Desktop\0807")
for path_name, dir, files_name in os.walk(os.getcwd()):
    for file in files_name:
        if str1 in file:
            file_list.append(os.path.join(path_name, file))
            
print(len(file_list))
print('\n'.join(file_list))



import pandas as pd
df_group = pd.DataFrame(columns = ['Path','Columns','False','True'])
type1 = '.xls'
type2 = '.csv'
type3 = '.et'
output_path = 'G:\\'
for i in range(len(file_list)):
    if type1 in file_list[i]:
        df = pd.read_excel(file_list[i], encoding = "gbk")
    elif type2 in file_list[i]:
        df = pd.read_csv(file_list[i], encoding = 'gbk')
    elif '.zip' in file_list[i]:
        continue
    print(i ,end = ' ')
    df.columns = df.columns.str.strip()
    col_name = df.columns
    bool_S = df.duplicated().value_counts()
    if len(bool_S)==1:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':0}
    elif len(bool_S)==2:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':bool_S[1]}
grouped = df_group.groupby('Columns')
groupname_list = list(grouped.size().index)
print('以下'+str(len(groupname_list))+'种格式数据分类，保留流水、账户类别')
print(grouped.size())

78
C:\Users\ThinkPad\Desktop\0807\中信银行\中信银行账户信息.csv
C:\Users\ThinkPad\Desktop\0807\中信银行\石春朋6217732305792362.xlsx
C:\Users\ThinkPad\Desktop\0807\中国银行\中国银行人员信息.csv
C:\Users\ThinkPad\Desktop\0807\中国银行\中国银行关联子账户信息.csv
C:\Users\ThinkPad\Desktop\0807\中国银行\中国银行强制措施信息.csv
C:\Users\ThinkPad\Desktop\0807\中国银行\中国银行账户信息.csv
C:\Users\ThinkPad\Desktop\0807\中国银行\蒋亚波6217866200016444930.xlsx
C:\Users\ThinkPad\Desktop\0807\中国银行\陈林燕6216696200012497120.xlsx
C:\Users\ThinkPad\Desktop\0807\交通银行\6222620140027659562陈小平.xls
C:\Users\ThinkPad\Desktop\0807\交通银行\6222620170015280436石海涛.xls
C:\Users\ThinkPad\Desktop\0807\交通银行\交通银行股份有限公司人员信息.csv
C:\Users\ThinkPad\Desktop\0807\交通银行\交通银行股份有限公司账户信息.csv
C:\Users\ThinkPad\Desktop\0807\兴业银行\622908353122273813胡晶晶.xls
C:\Users\ThinkPad\Desktop\0807\兴业银行\622908493042182114石春朋.xls
C:\Users\ThinkPad\Desktop\0807\兴业银行\兴业银行股份有限公司人员信息.csv
C:\Users\ThinkPad\Desktop\0807\兴业银行\兴业银行股份有限公司账户信息.csv
C:\Users\ThinkPad\Desktop\0807\农业银行\中国农业银行人员信息.csv
C:\Users\ThinkPad\Desktop\0807\农业银行\中

In [4]:
group_流水 = grouped.get_group(groupname_list[0])
group_流水['文件序号I'] = group_流水.index

group_证件= grouped.get_group(groupname_list[2])

group_账户 = grouped.get_group(groupname_list[3])
group_账户['文件序号I'] = group_账户.index

# group_流水2 = grouped.get_group(groupname_list[1])
# group_流水2['文件序号I'] = group_流水2.index

# group_任务 = grouped.get_group(groupname_list[1])
group_任务 =pd.DataFrame()
 
with pd.ExcelWriter(output_path + "文件目录.xlsx") as writer:
    group_流水.to_excel(writer, sheet_name = '流水文件目录',index = None)
#     group_流水2.to_excel(writer, sheet_name ='流水文件目录-标准',index = None)
    group_账户.to_excel(writer, sheet_name ='账户文件目录',index = None)
print('文件目录Done!')


import numpy as np
import re
import datetime
df_忽略行 = pd.DataFrame()
df_冲正 = pd.DataFrame()
all_流水 = pd.DataFrame()
all_账户 = pd.DataFrame()
all_证件 = pd.DataFrame()
all_任务 = pd.DataFrame()
for i in range(len(file_list)):
    if file_list[i] in list(group_流水['Path']):
        if file_list[i] in list(group_流水['Path']):  #查控平台流水
            if type1 in file_list[i] or type3 in file_list[i]:
                df = pd.read_excel(file_list[i],encoding = "gbk",dtype = "object")
            elif type2 in file_list[i]:
                df = pd.read_csv(file_list[i], encoding = "gbk")
            df.columns = df.columns.str.strip()      # 去掉标题行的\t
            col_name = df.columns.tolist()
        print(i,end=' ')
        df['path'] = file_list[i]
        df['序号'] = df.index
        df = df.applymap(lambda x: re.sub('\t','',str(x)))  # 去掉每个元素的\t
        df['交易卡号'] = df['交易卡号'].map(lambda x: np.NaN if str(x)=='' else x)
        df['交易卡号'][df['交易卡号'].isnull()]=df['交易账号'][df['交易卡号'].isnull()] # 用账号填充卡号空白
        df_忽略行 = df_忽略行.append(df[(df['交易金额']=='') | (df['交易金额']=='交易金额') | (df['交易卡号']=='')| (df['交易时间']=='')])
        df = df.drop(df[(df['交易金额']=='') | (df['交易金额']=='交易金额') | (df['交易卡号']=='') | (df['交易时间']=='')].index)  # 记录后删除交易金额为空或者账号卡号都为空的无效流水行
        df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
        df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
        df['交易余额'] = df['交易余额'].map(lambda x: float(x))  #交易余额指定浮点型
        df['收付标志'] = df['收付标志'].map(lambda x:'出' if x=='借' else ('进' if x=='贷' else x))
        df = df.drop_duplicates(subset = ['交易卡号','交易时间','交易金额','交易余额','收付标志','交易对手账卡号'],keep = 'last')  #去除重复
        df = df.sort_values(by = ['交易卡号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:] # 时间升序排序更新index
        print(str(i)+'. 单表去重后记录数:'+str(df.shape[0]),end = ' ')
        df_负 = df[df['交易金额']<0]
        for j in range(0,len(df_负)):
            df_正 = df[(df['交易金额']== abs(df_负['交易金额'].iloc[j])) & \
                (df['收付标志']== df_负['收付标志'].iloc[j]) & \
                (df['交易时间']<= df_负['交易时间'].iloc[j]) & \
                (df['交易卡号']== df_负['交易卡号'].iloc[j]) & \
                (
                 (df['交易对手账卡号']== df_负['交易对手账卡号'].iloc[j]) | \
                 ((df['交易对手账卡号'] == '') & (df_负['交易对手账卡号'].iloc[j] == '')) | \
                 ((df['交易对手账卡号'] != '') & (df_负['交易对手账卡号'].iloc[j] == '')) | \
                 ((df['交易对手账卡号'] == '') & (df_负['交易对手账卡号'].iloc[j] != '')) | \
                 ((df['交易对手账卡号'] != '') & (df['交易卡号']==df_负['交易对手账卡号'].iloc[j]))
                )
                ]
            d_labels = [df_正.index.max(),df_负.index[j]]  # 一对冲正匹配记录的索引
            if len(df_正)>0:
                df_冲正 = df_冲正.append(df.loc[d_labels])
                df = df.drop(d_labels) # 记录后删除一对冲正匹配的无效流水行
                print(d_labels)
            elif len(df_正)==0:
                print(d_labels)
        print('剔除冲正后保留记录数:'+str(df.shape[0]))
        all_流水 = all_流水.append(df)
    elif file_list[i] in list(group_账户['Path']):
        if type1 in file_list[i]:
                df_account = pd.read_excel(file_list[i],encoding = "gbk",dtype = "object")
        elif type2 in file_list[i]:
                df_account = pd.read_csv(file_list[i], engine="python")
        df_account.columns = df_account.columns.str.strip()
        df_account['path'] = file_list[i]
        df_account = df_account.applymap(lambda x: re.sub('\t','',str(x)))
        all_账户 = all_账户.append(df_account)
    elif file_list[i] in list(group_证件['Path']):
        if type1 in file_list[i]:
            df_identity = pd.read_excel(file_list[i],encoding = "gbk")
        elif type2 in file_list[i]:
            df_identity = pd.read_csv(file_list[i], engine="python")
#         df_identity = df_identity.iloc[:,0:3]
#         df_identity.columns = ['开户名称','证件类型','证件号码']
        df_identity['path'] = file_list[i]
        df_identity = df_identity.applymap(lambda x: re.sub('\t','',str(x)))
        all_证件 = all_证件.append(df_identity)
#     elif file_list[i] in list(group_任务['Path']):
#         if type1 in file_list[i]:
#                 df_task = pd.read_excel(file_list[i],encoding = "gbk",dtype = "object")
#         elif type2 in file_list[i]:
#                 df_task = pd.read_csv(file_list[i], engine="python")
#         df_task.columns = df_task.columns.str.strip()
#         df_task['path'] = file_list[i]
#         df_task = df_task.applymap(lambda x: re.sub('\t','',str(x)))
#         all_任务 = all_任务.append(df_task)
        
        
all_流水.to_csv(output_path + "流水合并剔除冲正.csv",index = None)
print(all_流水.shape)
with pd.ExcelWriter(output_path + "剔除行.xlsx") as writer:
    df_冲正.to_excel(writer, sheet_name='冲正全',index = None)
    df_忽略行.to_excel(writer, sheet_name='忽略行',index = None)
print(df_冲正.shape,end = ' ')
print(df_忽略行.shape)

all_账户.to_excel(output_path + "账户合并.xlsx",index = None)
print(all_账户.shape)
all_证件.to_excel(output_path + "证件合并.xlsx",index = None)
print(all_证件.shape)

all_任务.to_excel(output_path + "任务合并.xlsx",index = None)

print('合并Done!')



# 卡号修正
# 汇总时已经填充了交易账号，如果交易卡号、交易账号是唯一对应关系，实际应该依赖交易账号填充对应的交易卡号
df = all_流水.sort_values(by = ['交易卡号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:]  # 流水汇总的all_流水 更新索引

# 手工微调
df['交易卡号'] = df['交易卡号'].str.split('_', 1,expand = True).iloc[:,0] 
df['交易卡号'] = df['交易卡号'].str.split('-', 1,expand = True).iloc[:,0]

df['交易卡号']= df['交易卡号'].replace('6230520400151611175','6230520400230644379')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6230520400151611175','6230520400230644379')
df['交易卡号']= df['交易卡号'].replace('6230520600111277575','6230520600115175270')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6230520600111277575','6230520600115175270')
df['交易卡号']= df['交易卡号'].replace('6228412834555392179','6228412834555080279')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228412834555392179','6228412834555080279')
df['交易卡号']= df['交易卡号'].replace('6228481338992122179','6228481339108759177')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228481338992122179','6228481339108759177')
df['交易卡号']= df['交易卡号'].replace('6228480848817180179','6228480848874876370')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228480848817180179','6228480848874876370')
df['交易卡号']= df['交易卡号'].replace('6217566400013513025','6217566400014268967')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6217566400013513025','6217566400014268967')
df['交易卡号']= df['交易卡号'].replace('6228480389555382479','6228480389558414071')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228480389555382479','6228480389558414071')
df['交易卡号']= df['交易卡号'].replace('6228412834553530978','6228412834553528873')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228412834553530978','6228412834553528873')
df['交易卡号']= df['交易卡号'].replace('6228482838833466775','6228482838835549172')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228482838833466775','6228482838835549172')

df['交易对手账卡号']= df['交易对手账卡号'].replace('1107095201003626697','6215581107004822894')




df = df[~((df['交易对手账卡号']=='6236681460012372435') & (df['对手户名']=='吴晶晶'))]
df = df[~((df['交易对手账卡号']=='6230780100017400236') & (df['对手户名']=='吴晶晶'))]
df = df[~((df['交易对手账卡号']=='6228411070343158413') & (df['对手户名']=='毛霓姣'))]
df = df[~((df['交易对手账卡号']=='6212261207008544728') & (df['对手户名']=='唐文强'))]
df = df[~((df['交易对手账卡号']=='6214830278991772') & (df['对手户名']=='傅洋'))]
df = df[~((df['交易对手账卡号']=='6228481259254490379') & (df['对手户名']=='孞凤芹'))]
df = df[~((df['交易对手账卡号']=='6214836124051679') & (df['对手户名']=='尹君东'))]
df = df[~((df['交易对手账卡号']=='6228480316044859064') & (df['对手户名']=='周志峰'))]
df = df[~((df['交易对手账卡号']=='6222031106003886704') & (df['对手户名']=='李宁江'))]
df = df[~((df['交易对手账卡号']=='6230366107007841602') & (df['对手户名']=='林克时'))]
df = df[~((df['交易对手账卡号']=='622908406965516613') & (df['对手户名']=='王琴'))]
df = df[~((df['交易对手账卡号']=='6226093711673251') & (df['对手户名']=='张彦青'))]
df = df[~((df['交易对手账卡号']=='6217003100015191884') & (df['对手户名']=='翁玉通'))]
df = df[~((df['交易对手账卡号']=='6222021208010967154') & (df['对手户名']=='曹文光'))]

df['交易卡号'] = df['交易卡号'].replace('6228480136729683476','6228480136756146876')
df['交易卡号'] = df['交易卡号'].replace('6212261207015636822','6222033901009025812')
df['交易卡号'] = df['交易卡号'].replace('6217231208010997215','6222031202009735910')
df['交易卡号'] = df['交易卡号'].replace('9080218010100100273702','6230361108126793851')
df['交易卡号'] = df['交易卡号'].replace('6212261208012220446','6222032407001751588')
df['交易卡号'] = df['交易卡号'].replace('9070722010100100629197','6230361107126113383')
df['交易卡号'] = df['交易卡号'].replace('101004974664358','6230910199035009196')
df['交易卡号'] = df['交易卡号'].replace('891425925200012','6231310401002624344')
df['交易户名'][df['交易卡号']=='6212261715009732589'] = df['交易户名'][df['交易卡号']=='6212261715009732589'].replace('','陈聪')
# df['交易户名'][df['交易卡号']=='6212261202035906396'] = df['交易户名'][df['交易卡号']=='6212261202035906396'].replace('','张一帆')
# df['交易户名'][df['交易卡号']=='6221804330000582705'] = df['交易户名'][df['交易卡号']=='6221804330000582705'].replace('','曾文强')
df = df[df['交易对手账卡号']!='62220340000024751625']
list1 = ['6228480848874876370','6230361107126113383','6215581304002803675','6217231207000527321','6222031408002003558',
         '6222032405004527799','6228480128545948076','6228410084509279575','6228480838935860877','6228481186713862676',
         '6212264100039282292', '6217001930006517016','6228480086799269872','6230580000119723844','6230361108126793851',
         '6230361108126968180','6212264100038892968','6227001935100184349','6227001935950338193','6227074756143888','6228480078248493377'] 
df = df[~df['交易卡号'].isin(list1)]

print('手工微调后',end = '')
print(df.shape)




df_账号 = df[(df['交易卡号']!=df['交易账号']) & (df['交易账号']!='')] #  提取交易卡号、交易账号的对应关系表
df_a1 = df_账号.groupby(['交易账号','交易卡号']).size().reset_index()
df_a2 = df_a1.groupby(['交易账号'],as_index = False).size()
card = df_a1[df_a1['交易账号'].isin(df_a2[df_a2==1].index)]
card.to_excel(output_path + "卡号账号唯一对应.xlsx",index = None)
print('账号卡号对应Done!')
dfs0 = pd.merge(df,card,how = 'left',on='交易账号')
print(dfs0.columns)
dfs0['交易卡号_x'][(dfs0['交易卡号_x']!=dfs0['交易卡号_y']) & (dfs0['交易卡号_y'].notnull())]=dfs0['交易卡号_y'][(dfs0['交易卡号_x']!=dfs0['交易卡号_y']) & (dfs0['交易卡号_y'].notnull())]
all_流水卡修正 = dfs0.iloc[:,:34]
all_流水卡修正.columns = all_流水.columns
all_流水卡修正.to_csv(output_path + "流水卡号.csv",index = None)
print(all_流水卡修正.shape)



# 户名填充
# 方法一：依赖账户开户信息表汇总的all_账户
df_账户 = all_账户[all_账户['账户开户名称'] != '']
df_账户['交易卡号'] = df_账户['交易卡号'].str.split('_', 1,expand = True).iloc[:,0]   # 账户开户信息中卡号去掉_后缀
#  执行了手动修正卡号则屏蔽
df_流水 = all_流水卡修正  #  df_流水 是手动修正卡号后的流水汇总 ，不需要手动就用账号卡号唯一关系修正的 all_流水卡修正 

dfs = pd.merge(df_流水,df_账户.drop_duplicates(subset='交易卡号'),how = 'left',on = '交易卡号').iloc[:,:(df_流水.shape[1]+2)]
dfs['交易户名'][dfs['账户开户名称'].notnull()] = dfs['账户开户名称'][dfs['账户开户名称'].notnull()]
dfs['交易证件号码'][dfs['开户人证件号码'].notnull()] = dfs['开户人证件号码'][dfs['开户人证件号码'].notnull()]
df = dfs.iloc[:,:34]
df.columns = df_流水.columns
print(df.shape)
# 方法二：依赖交易对手卡号、对手户名的唯一对应关系
df_户名 = df[(df['交易对手账卡号'].isin(df['交易卡号'][df['交易户名']==''])) & (df['对手户名']!='')]  # 卡号为主账号且作为对手卡号出现时对手户名非空，作为本方卡号出现时户名缺失
df_a1 = df_户名.groupby(['交易对手账卡号','对手户名']).size().reset_index()
df_a2 = df_a1.groupby(['交易对手账卡号'],as_index=False).size()
account2 = df_a1[df_a1['交易对手账卡号'].isin(df_a2[df_a2==1].index)]
account2.to_excel(output_path + "卡号户名唯一对应.xlsx",index = None)
print('卡号户名对应Done!')
dfs2 = pd.merge(df,account2,how = 'left',left_on='交易卡号',right_on='交易对手账卡号')
print(dfs2.columns)
dfs2['交易户名'][dfs2['对手户名_y'].notnull()] = dfs2['对手户名_y'][dfs2['对手户名_y'].notnull()]
df = dfs2.iloc[:,:34]
df.columns = df_流水.columns
print(df.shape)
# # 方法三：依赖手工整理的银行卡号、姓名唯一对应表格
# account3 = pd.read_excel('E:\\7.3徐州网赌\\银行卡人员信息整理(提供给公司).xls',sheet_name = '去重',encoding = 'gbk')
# dfs3 = pd.merge(df,account3.drop_duplicates(subset='银行卡号'),how = 'left',left_on='交易卡号',right_on='银行卡号')
# dfs3['交易户名'][dfs3['姓名'].notnull()] = dfs3['姓名'][dfs3['姓名'].notnull()]
# df = dfs3.iloc[:,:34]
# df.columns = df_流水.columns
# print(df.shape)
df_流水户名 = df
df_流水户名.to_csv(output_path +"流水户名.csv",index = None)
print('流水户名Done!')

#--------------------------------------------------保留7月以后---------------------------------------------------------#
df = df[(df['交易时间'].str[0:4].isin(['2020']) ) & (df['交易时间'].str[5:7].isin(['07','08','09']) )]
#--------------------------------------------------保留7月以后---------------------------------------------------------#

df_S1 = df.groupby('交易卡号').size().reset_index()
df_S2 = pd.DataFrame(columns = ['交易卡号','Size','False','True'])
df_S3 = pd.DataFrame(columns = ['交易卡号','交易户名','证件号码','夜间交易次数','最早时间','最晚时间','余额','总和','净和','入向','出向','入向对手数','出向对手数','收付检验起始值','Path'])
for i in df_S1.index:
    df_every = df.groupby('交易卡号').get_group(df_S1['交易卡号'][i])
    bool_S = df_every.duplicated(subset = ['交易卡号','交易时间','交易金额','交易余额','收付标志','交易对手账卡号','交易币种'],keep = 'last').value_counts()
    df_S2.loc[i] = {'交易卡号':df_every['交易卡号'].iloc[0],
                        'Size':df_every.shape[0],
                        'False':bool_S[0],
                        'True':bool_S.map(lambda x:0 if len(bool_S)==1 else bool_S[1]).values[0]
                    }
    df_every = df_every.drop_duplicates(subset = ['交易卡号','交易时间','交易金额','交易余额','收付标志','交易对手账卡号','交易币种'],keep = 'last')
    df_night = df_every[df_every['交易时间'].str[11:13].isin(['00','01','02','03','04','05','06'])]
    df_S3.loc[i] = {'交易卡号':df_every['交易卡号'].iloc[0],
                    '交易户名':df_every['交易户名'].iloc[0],
                    '证件号码':df_every['交易证件号码'].iloc[0],
                    '夜间交易次数':len(df_night),
                    '最早时间':df_every['交易时间'].min(),
                    '最晚时间':df_every['交易时间'].max(),
                    '余额':df_every['交易余额'].loc[df_every.index.max()],
                    '总和':df_every['交易金额'].sum(),
                    '净和':df_every['交易金额'][df_every['收付标志']=='进'].sum()-df_every['交易金额'][df_every['收付标志']=='出'].sum(),
                    '入向':df_every['交易金额'][df_every['收付标志']=='进'].sum(),
                    '出向':df_every['交易金额'][df_every['收付标志']=='出'].sum(),
                    '入向对手数':len(df_every['交易对手账卡号'][df_every['收付标志']=='进'].drop_duplicates()),
                    '出向对手数':len(df_every['交易对手账卡号'][df_every['收付标志']=='出'].drop_duplicates()),
                    '收付检验起始值':df_every['交易余额'].iloc[0]+df_every.apply(lambda x: x['交易金额'] if x['收付标志']=='出' else -x['交易金额'],axis=1).iloc[0],
                    'Path':'\n'.join(df_every['path'].drop_duplicates().tolist())
                   }
    print(i,end = ' ')
df_S = pd.merge(df_S3,df_S2,how = 'left',on = '交易卡号')
df_账户['开户网点'][df_账户['开户网点']==''] = df_账户['账号开户银行'][df_账户['开户网点']=='']
dfs_账户银行 = pd.merge(df_S,df_账户.drop_duplicates(subset='交易卡号'),how = 'left',on = '交易卡号')
df_S['开户网点'] = dfs_账户银行['开户网点']
df_S.to_excel(output_path + "交易统计.xlsx",index = None)
# 统计完了，按照六大要素+币种 删除重复项（因为手动改了-101与-102的双向交易的卡号后缀，-101出-102进这类记录会造成自实体链接，不影响）
all_流水完整 = df.drop_duplicates(subset = ['交易卡号','交易时间','交易金额','交易余额','收付标志','交易对手账卡号','交易币种'],keep = 'last')
all_流水完整.to_csv(output_path + "流水完整.csv",index = None)
print(all_流水完整.shape)

D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


文件目录Done!
22 22. 单表去重后记录数:1333 [84, 85]
[155, 156]
[1167, 1168]
[1171, 1172]
[1236, 1237]
[1239, 1240]
[1315, 1316]
剔除冲正后保留记录数:1319
26 26. 单表去重后记录数:684 剔除冲正后保留记录数:684
29 29. 单表去重后记录数:1723 [276, 277]
[278, 279]
[280, 281]
[282, 283]
[310, 311]
[355, 356]
[445, 446]
[447, 448]
[749, 750]
[1685, 1686]
剔除冲正后保留记录数:1703
30 30. 单表去重后记录数:3210 [53, 54]
[1994, 1995]
剔除冲正后保留记录数:3206
31 31. 单表去重后记录数:1946 [12, 13]
[14, 15]
[97, 98]
[119, 120]
[121, 122]
[144, 145]
[204, 205]
[624, 625]
[666, 667]
[1187, 1188]
[1480, 1481]
[1482, 1483]
剔除冲正后保留记录数:1922
33 33. 单表去重后记录数:2277 [397, 398]
[428, 429]
[430, 431]
[432, 433]
[434, 435]
[839, 840]
[845, 846]
[1119, 1120]
剔除冲正后保留记录数:2261
(11095, 35)
(78, 35) (0, 35)
(50, 19)
(36, 19)
合并Done!


D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


手工微调后(11095, 35)
账号卡号对应Done!
Index([    '交易卡号_x',       '交易账号',   '查询反馈结果原因',       '交易户名',     '交易证件号码',
             '交易时间',       '交易金额', 'Unnamed: 7',       '交易余额',       '收付标志',
          '交易对手账卡号',       '现金标志',       '对手户名',     '对手身份证号',     '对手开户银行',
             '摘要说明',       '交易币种',     '交易网点名称',       '交易场所',      '交易发生地',
           '交易是否成功',        '传票号',       'IP地址',      'MAC地址',     '对手交易余额',
            '交易流水号',         '渠道',        '日志号',       '凭证种类',        '凭证号',
            '交易柜员号',         '备注',       '交易类型',       'path',         '序号',
           '交易卡号_y',            0],
      dtype='object')


D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ValueError: Length mismatch: Expected axis has 34 elements, new values have 35 elements